In [ ]:
#! conda install spacy
#python -m spacy download en_core_web_sm

In [281]:
# Import the dependencies
import re 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# Set the maximum column width to 200. 
pd.set_option('max_colwidth', 200)

# Import spacy library
import spacy
# Load the small English language model for spacy
nlp = spacy.load("en_core_web_sm")

In [282]:
# Read the bbc_news_articles.csv file into a DataFrame.
d4_emails_df = pd.read_csv('../resources/scrubbed_data.csv')
# Display the first 20 rows. 
d4_emails_df.head()

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforcement is sending a growing message that these violations are not important…and that reckless behavior is not of great concern. Second item: affordable de...,"Good morning Ron, \n\nThank you for reaching out, and I apologize for the delayed response. Council Pro Tem Romero Campbell maintains regular communication with DPD District 3, which serves Southe...",Staff1,NaN,NaN
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to put a yellow flashing pedestrian light on Dahlia and the intersection of the Highline Canal.. Also is there aywya that a turn signal could be impleme...,Thank you for reaching out to our office. The area engineers have looked at this intersection where the High Line Canal crosses Dahlia multiple times at our request and have determined that a fla...,Staff2,NaN,NaN
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,"Accident took place at Happy Canyon & Jasmine. Last week an extremely careless driver drove at a high rate of speed and got stuck on the median island. Took out a tree, dug big holes in the ground...","Good morning TJ,\n\nThank you for reaching out to our office and for bringing this to our attention. I have reported this to the city via 311 and it will be directed to the parks and recreation te...",Staff1,NaN,NaN
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,"Has many concerns about getting a compost cart, had many issues with Solid Waste and the person they were talking to. Been trying to get a compost cart for 2 years since moving into their house. N...","Thank you for reaching out to our office and for your thoughtful email. I hope I can touch on all your points in my response.\nUnfortunately, it is not possible for you to pick up a compost bin. ...",Staff2,"Clara sent another email back with more questions about how to volunteer, wants a new rec center, and still wants more info on compost cart.",Did not respond as they were the same questions I had answewred previously
4,Pauline,NaN,pauline@email.com,0,2024-08-21,"At any rate my concern is how will HOA condo units be charges? $150.00 per unit per year so a 200 unit building would pay $40,000 additional on storm water bill. I look forward to clarification o...","Good afternoon Pauline, \n\nThank you for reaching out to our office. As you know the sidewalk fee in November 2022, a citizen-led ballot initiative passed by voters directed Denver's Department o...",Staff1,NaN,NaN


## Preprocessing

In [283]:
# Check for null values.
d4_emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 57 non-null     object
 1   affected_address     40 non-null     object
 2   email_address        57 non-null     object
 3   case_number          57 non-null     int64 
 4   date                 57 non-null     object
 5   constituent_email_1  57 non-null     object
 6   d4_response_1        57 non-null     object
 7   d4_staff_member      57 non-null     object
 8   constituent_email_2  11 non-null     object
 9   d4_response_2        12 non-null     object
dtypes: int64(1), object(9)
memory usage: 4.6+ KB


In [284]:
# Use Parts of Speech (POS) tagging to extract the nouns, adjectives, and verbs from the constituent_email_1 column.

# Create a function that takes a string and returns only the nouns and verbs.
def get_nouns_adjs_verbs(text: str) -> str:
    tokens = nlp(text)
    nouns_adjs_verbs = ' '.join([token.text for token in tokens if token.pos_ == "NOUN" or token.pos_ == "ADJ" or token.pos_ == "VERB"])
    unique = set(nouns_adjs_verbs.lower().split())
    return ' '.join(unique)
    
d4_emails_df['nouns_adjs_verbs'] = d4_emails_df['constituent_email_1'].apply(get_nouns_adjs_verbs)
d4_emails_df['nouns_adjs_verbs']    

0          sending concern affordable second item police great enforcement lack tax message wanting more denver violations behavior set important accomplish code goals presence reckless growing information
1                                                                                                       put pedestrian light is aywya signal implemented intersection turn like flashing yellow possible know
2     place got removal speed road report various spilled driver fluids person stuff big stuck escape week holes island is dug city truck took inform drove oversaw last imagine high ground trying fixes ...
3                                                                                                needs get compost many issues person house years has talking concerns message cart moving had getting trying
4     have general be unit units wage concern year budgets clarification inflation tax rules condo losing building lookup possible class made mentioned end scoop new middle reg

In [286]:
topic_dict = {
    'homeless': 'Homeless',
    'shelter': 'Homeless',
    'graffiti': 'Graffiti',
    'paint': 'Graffiti',
    'pothole': 'Pothole',
    'holes': 'Pothole',
    'animal': 'Animal',
    'pest': 'Animal',
    'weeds': 'Vegitation',
    'trees': 'Vegitation',
    'overgrown': 'Vegitation',
    'neighborhood': 'Neighborhood',
    'hoa': 'Neighborhood',
    'rno': 'Neighborhood',
    'sidewalk': 'Neighborhood',
    'sidewalks': 'Neighborhood',
    'fence': 'Neighborhood',
    'fences': 'Neighborhood',
    'snow': 'Snow Removal',
    'ice': 'Snow Removal',
    'plows': 'Snow Removal',
    'vehicle': 'Vehicle',
    'vehicles': 'Vehicle',
    'car': 'Vehicle',
    'cars': 'Vehicle',
    'motorcycle': 'Vehicle',
    'motorcycles': 'Vehicle',
    'parking': 'Parking',
    'police': 'Police',
    'gang': 'Police',
    'loud': 'Police',
    'drugs': 'Police',
    'crime': 'Police',
    'firework': 'Fireworks',
    'dumping': 'Dumping',
    'trash': 'Trash',
    'garbage': 'Trash',
    'compost': 'Trash',
    'recycling': 'Trash',
    'rent': 'Housing',
    'rental': 'Housing',
    'rentals': 'Housing',
    'apartment': 'Housing',
    'apartments': 'Housing',
    'policy': 'Policy',
    'tax': 'Policy',
    'taxes': 'Policy',
    'mayor': 'Policy',
    'council': 'Policy',
    'environmental': 'Policy',
    'environment': 'Policy',
    'rezoning': 'Policy',
    'rezone': 'Policy',
    'government': 'Policy',
    'racing': 'Street Racing',
    'transit': 'Transit',
    'traffic': 'Transit',
    'pedestrian': 'Transit',
    'intersection': 'Transit',
    'bicycle': 'Transit',
    'bike': 'Transit',
    'speed': 'Transit',
    'pavement': 'Transit',
    'parks': 'Parks',
    'park': 'Parks',
    'playground': 'Parks',
    'trails': 'Parks',
    'pool': 'Parks',
    'gym': 'Parks',
}

# Function to check if any key in dict is in the string
def check_topics(value, topic_dict):
    topics_set = set()
    for key in topic_dict:
        words = value.split()
        if key in words:
            topics_set.add(topic_dict[key])
    
    if len(topics_set) > 0:
        return ','.join(topics_set)
    return 'Other'

# Apply the function and create a new column
d4_emails_df['topics'] = d4_emails_df['nouns_adjs_verbs'].apply(lambda x: check_topics(x, topic_dict))
d4_emails_df.head()

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2,nouns_adjs_verbs,topics
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforcement is sending a growing message that these violations are not important…and that reckless behavior is not of great concern. Second item: affordable de...,"Good morning Ron, \n\nThank you for reaching out, and I apologize for the delayed response. Council Pro Tem Romero Campbell maintains regular communication with DPD District 3, which serves Southe...",Staff1,NaN,NaN,sending concern affordable second item police great enforcement lack tax message wanting more denver violations behavior set important accomplish code goals presence reckless growing information,"Police,Policy"
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to put a yellow flashing pedestrian light on Dahlia and the intersection of the Highline Canal.. Also is there aywya that a turn signal could be impleme...,Thank you for reaching out to our office. The area engineers have looked at this intersection where the High Line Canal crosses Dahlia multiple times at our request and have determined that a fla...,Staff2,NaN,NaN,put pedestrian light is aywya signal implemented intersection turn like flashing yellow possible know,Transit
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,"Accident took place at Happy Canyon & Jasmine. Last week an extremely careless driver drove at a high rate of speed and got stuck on the median island. Took out a tree, dug big holes in the ground...","Good morning TJ,\n\nThank you for reaching out to our office and for bringing this to our attention. I have reported this to the city via 311 and it will be directed to the parks and recreation te...",Staff1,NaN,NaN,place got removal speed road report various spilled driver fluids person stuff big stuck escape week holes island is dug city truck took inform drove oversaw last imagine high ground trying fixes ...,"Pothole,Transit"
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,"Has many concerns about getting a compost cart, had many issues with Solid Waste and the person they were talking to. Been trying to get a compost cart for 2 years since moving into their house. N...","Thank you for reaching out to our office and for your thoughtful email. I hope I can touch on all your points in my response.\nUnfortunately, it is not possible for you to pick up a compost bin. ...",Staff2,"Clara sent another email back with more questions about how to volunteer, wants a new rec center, and still wants more info on compost cart.",Did not respond as they were the same questions I had answewred previously,needs get compost many issues person house years has talking concerns message cart moving had getting trying,Trash
4,Pauline,NaN,pauline@email.com,0,2024-08-21,"At any rate my concern is how will HOA condo units be charges? $150.00 per unit per year so a 200 unit building would pay $40,000 additional on storm water bill. I look forward to clarification o...","Good afternoon Pauline, \n\nThank you for reaching out to our office. As you know the sidewalk fee in November 2022, a citizen-led ballot initiative passed by voters directed Denver's Department o...",Staff1,NaN,NaN,have general be unit units wage concern year budgets clarification inflation tax rules condo losing building lookup possible class made mentioned end scoop new middle regs folks bill tool fee pay ...,"Neighborhood,Policy"


In [247]:
# Remove numbers and non-alphabetic characters from the constituent_email_1 column.
d4_topic_df['constituent_email_1'] = d4_emails_df['constituent_email_1'].apply(lambda x: re.sub(r'[^a-zA-Z\s ]', '', str(x)))
d4_topic_df['constituent_email_1']

0          sending concern affordable second item police great enforcement lack tax message wanting more denver violations behavior set important accomplish code goals presence reckless growing information
1                                                                                                       put pedestrian light is aywya signal implemented intersection turn like flashing yellow possible know
2     place got removal speed road report various spilled driver fluids person stuff big stuck escape week holes island is dug city truck took inform drove oversaw last imagine high ground trying fixes ...
3                                                                                                needs get compost many issues person house years has talking concerns message cart moving had getting trying
4     have general be unit units wage concern year budgets clarification inflation tax rules condo losing building lookup possible class made mentioned end scoop new middle reg

## Process the Text to Tokens and Counts.

In [248]:
# Create an instance of the CountVectorizer and set the max_df to 0.95 and min_df to 5, and use the "english" stopwords.
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words='english')
cv

CountVectorizer(max_df=0.95, min_df=3, stop_words='english')

In [249]:
# Transform each row from the news_summary to a DTM.
dtm = cv.fit_transform(d4_topic_df['constituent_email_1'])
# Get the shape of the DTM.
print(dtm.shape)

(57, 147)


## LDA

In [250]:
# Create and instance of the LatentDirichletAllocation() class with 5 topics.
LDA = LatentDirichletAllocation(n_components=15,random_state=42)
# Fit the model with our DTM data.
LDA_data = LDA.fit(dtm)
# Check the length of the vocabulary 
len(cv.get_feature_names_out())

147

## Get the Top 15 Words Per Topic

In [251]:
# Print the top 15 words for each topic.
for index,topic in enumerate(LDA.components_):
    print(f'The Top 10 Words For Topic #{index+1}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

The Top 10 Words For Topic #1
['residents', 'having', 'parking', 'car', 'mayor', 'things', 'neighbors', 'know', 'issues', 'increased', 'week', 'need', 'hoa', 'neighborhood', 'street']


The Top 10 Words For Topic #2
['situation', 'information', 'little', 'stop', 'safe', 'better', 'years', 'police', 'nice', 'good', 'addressed', 'time', 'racing', 'getting', 'street']


The Top 10 Words For Topic #3
['park', 'thank', 'years', 'issues', 'information', 'intersection', 'street', 'look', 'email', 'going', 'police', 'neighborhood', 'concern', 'property', 'sidewalk']


The Top 10 Words For Topic #4
['new', 'multiple', 'going', 'lane', 'safer', 'planning', 'help', 'complex', 'writing', 'said', 'live', 'corner', 'driving', 'concerned', 'need']


The Top 10 Words For Topic #5
['appreciate', 'better', 'turn', 'thank', 'intersection', 'message', 'second', 'cross', 'lack', 'great', 'regarding', 'like', 'idea', 'community', 'use']


The Top 10 Words For Topic #6
['lanes', 'concerned', 'noticed', 'mean

### **Question:** What is the label for each topic? 
---
- TOPIC 1: Entertainment
- TOPIC 2: Sports
- TOPIC 3: Business
- TOPIC 4: Politics
- TOPIC 5: Technology

## Assign the Topics and Labels to the News Summaries

In [92]:
# Transform our DTM so we get an array with the (number_of_documents, number_of_topics).
topic_results = LDA.transform(dtm)

# Get the shape of the topic results
topic_results.shape

(57, 5)

In [93]:
# Read the bbc_news_articles.csv file into a DataFrame.
d4_emails_df = pd.read_csv('../resources/scrubbed_data.csv')
# Display the DataFrame. 
d4_emails_df.head()

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforcement is sending a growing message that these violations are not important…and that reckless behavior is not of great concern. Second item: affordable de...,"Good morning Ron, \n\nThank you for reaching out, and I apologize for the delayed response. Council Pro Tem Romero Campbell maintains regular communication with DPD District 3, which serves Southe...",Staff1,NaN,NaN
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to put a yellow flashing pedestrian light on Dahlia and the intersection of the Highline Canal.. Also is there aywya that a turn signal could be impleme...,Thank you for reaching out to our office. The area engineers have looked at this intersection where the High Line Canal crosses Dahlia multiple times at our request and have determined that a fla...,Staff2,NaN,NaN
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,"Accident took place at Happy Canyon & Jasmine. Last week an extremely careless driver drove at a high rate of speed and got stuck on the median island. Took out a tree, dug big holes in the ground...","Good morning TJ,\n\nThank you for reaching out to our office and for bringing this to our attention. I have reported this to the city via 311 and it will be directed to the parks and recreation te...",Staff1,NaN,NaN
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,"Has many concerns about getting a compost cart, had many issues with Solid Waste and the person they were talking to. Been trying to get a compost cart for 2 years since moving into their house. N...","Thank you for reaching out to our office and for your thoughtful email. I hope I can touch on all your points in my response.\nUnfortunately, it is not possible for you to pick up a compost bin. ...",Staff2,"Clara sent another email back with more questions about how to volunteer, wants a new rec center, and still wants more info on compost cart.",Did not respond as they were the same questions I had answewred previously
4,Pauline,NaN,pauline@email.com,0,2024-08-21,"At any rate my concern is how will HOA condo units be charges? $150.00 per unit per year so a 200 unit building would pay $40,000 additional on storm water bill. I look forward to clarification o...","Good afternoon Pauline, \n\nThank you for reaching out to our office. As you know the sidewalk fee in November 2022, a citizen-led ballot initiative passed by voters directed Denver's Department o...",Staff1,NaN,NaN


In [94]:
# Use the add_topic_labels function to add the topic and topic label to each news summary. 
# Dictionary of  topics and topic label.
topic_labels = {
    1: 'Traffic,Trees',
    2: 'Homeless',
    3: 'Street Racing',
    4: 'Parking,Parks',
    5: 'Trash,Parks'
}

# Define the function and pass in the DataFrame, the topic_results, and topic_labels dictionary.
def add_topic_labels(df, topic_results, topic_labels):
    # Find the dominant topic for each document and add the label to a new column
    df['topic'] = topic_results.argmax(axis=1) + 1
    # Use the map function to add the topic label to the news summary based on the topic number.
    df['topic_label'] = df['topic'].map(topic_labels)


In [95]:
# Call the function to add topic labels to your DataFrame.
add_topic_labels(d4_emails_df, topic_results, topic_labels)

In [96]:
# Display the first 20 rows of the updated DataFrame. 
d4_emails_df.head(20)

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2,topic,topic_label
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforcement is sending a growing message that these violations are not important…and that reckless behavior is not of great concern. Second item: affordable de...,"Good morning Ron, \n\nThank you for reaching out, and I apologize for the delayed response. Council Pro Tem Romero Campbell maintains regular communication with DPD District 3, which serves Southe...",Staff1,NaN,NaN,3,Street Racing
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to put a yellow flashing pedestrian light on Dahlia and the intersection of the Highline Canal.. Also is there aywya that a turn signal could be impleme...,Thank you for reaching out to our office. The area engineers have looked at this intersection where the High Line Canal crosses Dahlia multiple times at our request and have determined that a fla...,Staff2,NaN,NaN,1,"Traffic,Trees"
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,"Accident took place at Happy Canyon & Jasmine. Last week an extremely careless driver drove at a high rate of speed and got stuck on the median island. Took out a tree, dug big holes in the ground...","Good morning TJ,\n\nThank you for reaching out to our office and for bringing this to our attention. I have reported this to the city via 311 and it will be directed to the parks and recreation te...",Staff1,NaN,NaN,1,"Traffic,Trees"
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,"Has many concerns about getting a compost cart, had many issues with Solid Waste and the person they were talking to. Been trying to get a compost cart for 2 years since moving into their house. N...","Thank you for reaching out to our office and for your thoughtful email. I hope I can touch on all your points in my response.\nUnfortunately, it is not possible for you to pick up a compost bin. ...",Staff2,"Clara sent another email back with more questions about how to volunteer, wants a new rec center, and still wants more info on compost cart.",Did not respond as they were the same questions I had answewred previously,1,"Traffic,Trees"
4,Pauline,NaN,pauline@email.com,0,2024-08-21,"At any rate my concern is how will HOA condo units be charges? $150.00 per unit per year so a 200 unit building would pay $40,000 additional on storm water bill. I look forward to clarification o...","Good afternoon Pauline, \n\nThank you for reaching out to our office. As you know the sidewalk fee in November 2022, a citizen-led ballot initiative passed by voters directed Denver's Department o...",Staff1,NaN,NaN,2,Homeless
5,Debbie,4504 S Yosemite St #361,debbie@email.com,0,2024-08-21,What actions if any is the City contemplating on making the downtown Denver the safe environment it was for so many years? I hear the Mayor talking about revitalizing downtown but have not seen an...,Thank you for reaching out to our office with your follow up questions from the HOA meeting. The Mayor is working with the Downtown Denver Partnership to revitalize the 16th Street Mall with the ...,Staff2,NaN,NaN,5,"Trash,Parks"
6,Michelle,NaN,michelle@email.com,0,2024-08-25,Were there any upgrades or specialty projects leftover from your predecessor’s time as councilwoman? The Bible Park playground is a hit and a wonderful legacy for Ms. Black. Was there anything n...,"Thank you for reaching out to our office. Many of the projects that were in the works before the previous Councilwoman retired are still being completed. These include, tunnels for an underpass ...",Staff2,NaN,NaN,2,Homeless
7,Joshua,Hampden Heights,joshua@email.com,0,2024-08-26,Bug infestation problem at apartment complex in Hampden Heights. The roaches are back and while they have yet to affect the health of my 3 year old or 1 year o

In [15]:
# Display the last 20 rows of the updated DataFrame.
d4_emails_df.tail(10)

,news_summary,topic,topic_label
2215,The new book claims Mr Prescott hosted a dinner in November 2003 where the prime minister told Mr Brown he would stand down before the next election because he had lost trust over the Iraq war.Mr ...,4,Politics
2216,"Actor Daniel Day-Lewis is to be presented with an award for his career in film at the Berlin Film Festival.Day-Lewis has competed four times at the Berlin Film Festival, with films In The Name Of ...",1,Entertainment
2217,"With so much time spent in the spotlight, Holmes has increasingly dropped hints that her ambition on the track has begun to wilt.Four months later, Holmes stormed to double Olympic gold and has si...",2,Sports
2218,"US oil prices have fallen by 6%, driven down by forecasts of a mild winter in the densely populated northeast.Statistics released last week showed that stockpiles of oil products in the US had ris...",3,Business
2219,"Instead, said Mr Doctorow, DRM systems were intended to control the group that electronics firms have most hold over - consumers.By setting up the alliance to work on a common control system, the ...",5,Technology
2220,"The Votes at 16 alliance said it was a good thing to ""engage people"" by lowering the candidacy age but argued lowering the voting age would be much more effective.That is because the age of majori...",4,Politics
2221,"West Ham boss Alan Pardew said: ""It's a shame because I thought there was good English banter in the crowd.The FA is to take action after trouble marred Wednesday's Carling Cup tie between Chelsea...",1,Entertainment
2222,"Navratilova, who made a comeback after retiring in 1994, will play doubles and mixed doubles events in 2005.I'm still that good.""Navratilova has won three Grand Slam mixed doubles titles since she...",2,Sports
2223,"""You would think they would have been available in December as it can take two months for the body to respond.""The first chance we will get to do that is at the players meeting on the Saturday bef...",2,Sports
2224,"Other than its warning on possible tax hikes, the NIESR report was optimistic about the state of the UK and global economy.The UK government will have to raise taxes or rein in spending if it want...",3,Business


**Question:** Did LDA do a good job at assigning the appropriate topic to the news summaries? 

**Answer:** Yes. Most of the news summaries look like they have been appropriately assigned the correct topic and topic label.